In [76]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Veriyi yükleme ve ön işleme
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")

df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Eğitim ve test veri setlerini ayırma
train_size = int(len(df) * 0.80)
test_size = len(df) - train_size
train, test = df[0:train_size], df[train_size:len(df)]

# Özellikleri oluşturma
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

df = create_features(df)
train = create_features(train)
test = create_features(test)

FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear']
TARGET = 'İstanbul günlük tüketim(m³/gün)'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Veriyi normalize etme
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

# LSTM için veriyi yeniden şekillendirme
X_train = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))

# LSTM modelini oluşturma
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Erken durdurma tanımlama
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=2, shuffle=False, callbacks=[early_stopping])

# Test verileri üzerinde tahmin yapma
y_pred = model.predict(X_test)

# Tahminleri orijinal ölçeğe döndürme
y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_orig = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

# MSE ve RMSE değerlerini hesaplama
mse = mean_squared_error(y_test_orig, y_pred_orig)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE): {:.4f}".format(mse))
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))

# MAE hesaplama
mae = mean_absolute_error(y_test_orig, y_pred_orig)
print("Mean Absolute Error (MAE):", mae)

# MAPE hesaplama
mape = mean_absolute_percentage_error(y_test_orig, y_pred_orig) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

Epoch 1/100
118/118 - 2s - loss: 0.0385 - val_loss: 0.0180 - 2s/epoch - 15ms/step
Epoch 2/100
118/118 - 0s - loss: 0.0202 - val_loss: 0.0127 - 241ms/epoch - 2ms/step
Epoch 3/100
118/118 - 0s - loss: 0.0166 - val_loss: 0.0115 - 242ms/epoch - 2ms/step
Epoch 4/100
118/118 - 0s - loss: 0.0154 - val_loss: 0.0110 - 252ms/epoch - 2ms/step
Epoch 5/100
118/118 - 0s - loss: 0.0148 - val_loss: 0.0106 - 258ms/epoch - 2ms/step
Epoch 6/100
118/118 - 0s - loss: 0.0142 - val_loss: 0.0100 - 259ms/epoch - 2ms/step
Epoch 7/100
118/118 - 0s - loss: 0.0136 - val_loss: 0.0093 - 257ms/epoch - 2ms/step
Epoch 8/100
118/118 - 0s - loss: 0.0129 - val_loss: 0.0087 - 261ms/epoch - 2ms/step
Epoch 9/100
118/118 - 0s - loss: 0.0123 - val_loss: 0.0081 - 265ms/epoch - 2ms/step
Epoch 10/100
118/118 - 0s - loss: 0.0117 - val_loss: 0.0076 - 261ms/epoch - 2ms/step
Epoch 11/100
118/118 - 0s - loss: 0.0112 - val_loss: 0.0072 - 258ms/epoch - 2ms/step
Epoch 12/100
118/118 - 0s - loss: 0.0107 - val_loss: 0.0069 - 262ms/epoch - 

In [77]:
LstmYPRED= y_pred_orig

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

# Veriyi yükleme ve ön işleme
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")

df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Eğitim ve test veri setlerini ayırma
train_size = int(len(df) * 0.80)
test_size = len(df) - train_size
train, test = df[0:train_size], df[train_size:len(df)]

# Özellikleri oluşturma
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

df = create_features(df)
train = create_features(train)
test = create_features(test)

FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear']
TARGET = 'İstanbul günlük tüketim(m³/gün)'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Veriyi normalize etme
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

# MLPRegressor modelini oluşturma ve eğitme
mlp_reg = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200, random_state=42)
mlp_reg.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapma
y_pred = mlp_reg.predict(X_test)

# Tahminleri orijinal ölçeğe döndürme
y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_orig = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

# MSE ve RMSE değerlerini hesaplama
mse = mean_squared_error(y_test_orig, y_pred_orig)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE): {:.4f}".format(mse))
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))

# MAE hesaplama
mae = mean_absolute_error(y_test_orig, y_pred_orig)
print("Mean Absolute Error (MAE):", mae)

# MAPE hesaplama
mape = mean_absolute_percentage_error(y_test_orig, y_pred_orig) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)


Mean Squared Error (MSE): 0.0024
Root Mean Squared Error (RMSE): 0.0485
Mean Absolute Error (MAE): 0.038298088621056846
Mean Absolute Percentage Error (MAPE): 0.3716887542262685


In [79]:
MLPRegressorYPRED= y_pred_orig

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# Veri setini yükleme ve hazırlama
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")
df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Eğitim ve test veri setlerini ayırma
train_size = int(len(df) * 0.80)
test_size = len(df) - train_size
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Özellikleri oluşturma
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

df = create_features(df)
train = create_features(train)
test = create_features(test)

FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear']
TARGET = 'İstanbul günlük tüketim(m³/gün)'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Veriyi normalize etme
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

# SVR modelini oluşturma ve eğitme
svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_model.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapma
y_pred = svr_model.predict(X_test)

# Tahminleri orijinal ölçeğe döndürme
y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_orig = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

# MSE ve RMSE değerlerini hesaplama
mse = mean_squared_error(y_test_orig, y_pred_orig)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE): {:.4f}".format(mse))
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))

# MAE hesaplama
mae = mean_absolute_error(y_test_orig, y_pred_orig)
print("Mean Absolute Error (MAE): {:.4f}".format(mae))

# MAPE hesaplama
mape = mean_absolute_percentage_error(y_test_orig, y_pred_orig) * 100
print("Mean Absolute Percentage Error (MAPE): {:.4f}".format(mape))


Mean Squared Error (MSE): 0.0027
Root Mean Squared Error (RMSE): 0.0521
Mean Absolute Error (MAE): 0.0416
Mean Absolute Percentage Error (MAPE): 0.4021


In [81]:
SVMYPRED= y_pred_orig

In [82]:
pred_final = (LstmYPRED+MLPRegressorYPRED+SVMYPRED)/3.0
print(mean_squared_error(y_test_orig, pred_final))

0.0017945344442359473
